In [2]:
import pandas as pd

In [50]:
df = pd.read_csv('oversample_train_df.csv')

In [51]:
df = df.drop(['text_en', 'date'], axis=1)

In [52]:
df.head(3)

,text,amusement_parks,ferris_wheels,water_parks,miniature_parks,baths_and_saunas,climbing,stadiums,winter_sports,natural_springs,...,local_museums,national_museums,fashion_museums,planetariums,zoos,aquariums,skyscrapers,towers,historic_architecture,bridges
0,"Szukam miejsc, w których mogę zrelaksować się...",2,3,2,2,3,2,2,2,4,...,4,4,3,3,3,3,2,4,4,4
1,Moje zainteresowania leżą w literaturze i sztu...,1,2,1,2,3,1,1,1,2,...,4,5,3,3,2,2,2,3,5,3
2,"Lubię także odwiedzać miejsca w sercu natury, ...",1,3,2,2,2,1,1,1,2,...,4,5,2,1,1,2,2,4,5,3


In [ ]:
! pip install pyMorfologik

In [ ]:
# https://github.com/dmirecki/pyMorfologik

In [13]:
! pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32898 sha256=4088ac15d508d55e0d81f2876c28e575d0f9d675434a23c8920945cf1c96fb38
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [73]:
from nltk.stem.snowball import SnowballStemmer
import re
from string import punctuation
import nltk
import spacy
from stop_words import get_stop_words
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser
import string

parser = ListParser()
stemmer = Morfologik()

stopwords_pl = get_stop_words("pl")

In [74]:
def preprocess_text(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    new_text = text.translate(translator)
    new_text = re.sub(r'\d+', '', new_text)
    new_text = re.sub(r'\s+', ' ', new_text)
    new_text = new_text.strip()
    new_text = new_text.lower()

    stems = stemmer.stem([new_text], parser)
    tokens = [(list(stems[i][1].keys())[0] if len(list(stems[i][1].keys())) > 0 else stems[i][0]) for i in range(len(stems))]

    filtered_tokens = [token for token in tokens if token not in stopwords_pl]
    filtered_tokens = [token for token in filtered_tokens if token!= '']
    processed_text = " ".join(filtered_tokens)

    return processed_text

In [69]:
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

parser = ListParser()
stemmer = Morfologik()
stems = stemmer.stem(['Ala ma kota'], parser)
stems

[('Ala',
  {'Ala': ['subst:sg:nom:f'],
   'Al': ['subst:sg:acc:m1+subst:sg:gen:m1'],
   'Alo': ['subst:sg:acc:m1+subst:sg:gen:m1']}),
 ('ma',
  {'mieć': ['verb:fin:sg:ter:imperf:refl.nonrefl'],
   'mój': ['adj:sg:nom.voc:f:pos']}),
 ('kota', {'kota': ['subst:sg:nom:f'], 'kot': ['subst:sg:acc:m1']})]

In [46]:
x = list(stems[0][1].keys())[0] if len(list(stems[0][1].keys())) > 0 else stems[0][0]
x

'wiedźmiński'

In [53]:
sample_text = df['text'][0]

In [54]:
sample_text

' Szukam miejsc, w których mogę zrelaksować się wokół pięknych krajobrazów, a jednocześnie cieszyć się wyjątkowymi zabytkami i tradycjami.. Jestem głównie zainteresowany przyrodą, architekturą i kulturą lokalną.Jestem turystą, który uwielbia spokojne miejsca z dala od zgiełku'

In [63]:
translator = str.maketrans("", "", punctuation)
new_text = sample_text.translate(translator)
new_text = re.sub(r'\d+', '', new_text)
new_text = re.sub(r'\s+', ' ', new_text)
new_text = new_text.strip()
new_text = new_text.lower()

In [64]:
new_text

'szukam miejsc w których mogę zrelaksować się wokół pięknych krajobrazów a jednocześnie cieszyć się wyjątkowymi zabytkami i tradycjami jestem głównie zainteresowany przyrodą architekturą i kulturą lokalnąjestem turystą który uwielbia spokojne miejsca z dala od zgiełku'

In [65]:
stems = stemmer.stem([new_text], parser)

In [66]:
stems

[('szukam', {'szukać': ['verb:fin:sg:pri:imperf:refl.nonrefl']}),
 ('miejsc', {'miejsce': ['subst:pl:gen:n2']}),
 ('w', {'w': ['prep:acc:nwok+prep:loc:nwok'], 'wiek': ['brev:pun']}),
 ('których',
  {'który': ['adj:pl:acc:m1.p1:pos+adj:pl:gen:m1.m2.m3.f.n1.n2.p1.p2.p3:pos+adj:pl:loc:m1.m2.m3.f.n1.n2.p1.p2.p3:pos']}),
 ('mogę', {'móc': ['verb:fin:sg:pri:imperf:nonrefl']}),
 ('zrelaksować',
  {'zrelaksować': ['verb:inf:perf:refl+verb:inf:perf:refl.nonrefl']}),
 ('się', {'się': ['siebie:acc:nakc+siebie:gen:nakc+subst:sg:nom:n2']}),
 ('wokół', {'wokół': ['adv+prep:gen']}),
 ('pięknych',
  {'piękny': ['adj:pl:acc:m1.p1:pos+adj:pl:gen:m1.m2.m3.f.n1.n2.p1.p2.p3:pos+adj:pl:loc:m1.m2.m3.f.n1.n2.p1.p2.p3:pos'],
   'piękne': ['subst:pl:gen:n2+subst:pl:loc:n2']}),
 ('krajobrazów', {'krajobraz': ['subst:pl:gen:m3']}),
 ('a', {'a': ['conj+interj+prep:nom+qub']}),
 ('jednocześnie', {'jednocześnie': ['adv:pos']}),
 ('cieszyć', {'cieszyć': ['verb:inf:imperf:refl.nonrefl']}),
 ('się', {'się': ['siebie:ac

In [67]:
len(stems)

35

In [68]:
tokenized_text = [(list(stems[i][1].keys())[0] if len(list(stems[i][1].keys())) > 0 else stems[i][0]) for i in range(len(stems))]
tokenized_text


['szukać',
 'miejsce',
 'w',
 'który',
 'móc',
 'zrelaksować',
 'się',
 'wokół',
 'piękny',
 'krajobraz',
 'a',
 'jednocześnie',
 'cieszyć',
 'się',
 'wyjątkowy',
 'zabytek',
 'i',
 'tradycja',
 'być',
 'głównie',
 'zainteresowany',
 'przyroda',
 'architektura',
 'i',
 'kultura',
 'lokalnąjestem',
 'turysta',
 'który',
 'uwielbiać',
 'spokojny',
 'miejsce',
 'z',
 'dala',
 'od',
 'zgiełk']

In [77]:
preprocess_text(df['text'][0])

'szukać miejsce w który móc zrelaksować wokół piękny krajobraz a jednocześnie cieszyć wyjątkowy zabytek i tradycja głównie zainteresowany przyroda architektura i kultura lokalny turysta który uwielbiać spokojny miejsce z dala zgiełk'

In [80]:
prep_texts = [preprocess_text(df['text'][i]) for i in range(len(df))]
df['prep_text'] = prep_texts
df[['text', 'prep_text']]

,text,prep_text
0,"Szukam miejsc, w których mogę zrelaksować się...",szukać miejsce w który móc zrelaksować wokół p...
1,Moje zainteresowania leżą w literaturze i sztu...,zainteresowanie leż w literatura i sztuca któr...
2,"Lubię także odwiedzać miejsca w sercu natury, ...",lubić także odwiedzać miejsce w serce natura a...
3,Skupiamy się na zapewnianiu dzieciom przyjemny...,skupiać zapewniać dziecko przyjemny doświadcze...
4,.Jestem fanem nowych technologii i uwielbiam o...,fan nowy technologia i uwielbiać odkrywać miej...
...,...,...
2295,"Będąc spokojną osobą, czerpię wielką przyjemno...",spokojny osoba czerpać wielki przyjemność z ha...
2296,.Mam powinowactwo do przyrody i aktywne zajęci...,mama powinowactwo przyroda i aktywny zajęcia r...
2297,Jestem ekologiem środowiskowym zainteresowanym...,ekolog środowiskowy zainteresowany zrównoważon...
2298,".Lubię zarówno wiedzę naukową, jak i emocje zw...",lubić zarówno wiedza naukowy i emocje związać ...


In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['prep_text'])
print(vectors.shape)

(2300, 2696)


In [86]:
print(vectors[0])

  (0, 2570)	0.2780523874922535
  (0, 256)	0.27624846524719654
  (0, 1919)	0.17854719091559004
  (0, 2223)	0.16389112192276592
  (0, 2124)	0.17961819858989864
  (0, 857)	0.11759105728130813
  (0, 806)	0.10801655207305173
  (0, 47)	0.15155078262323338
  (0, 1651)	0.16270931104897376
  (0, 2476)	0.1871830593468074
  (0, 525)	0.23021989724125266
  (0, 2096)	0.16469436496700166
  (0, 2454)	0.18533592019384595
  (0, 2362)	0.21977264282501247
  (0, 206)	0.1971863367076258
  (0, 649)	0.22036694434495008
  (0, 770)	0.20281453890031417
  (0, 1344)	0.23802371439657233
  (0, 2309)	0.2597169738034561
  (0, 2614)	0.2979860597227402
  (0, 1007)	0.17255129488326426
  (0, 789)	0.2010520331022472
  (0, 928)	0.16787874515101442
  (0, 2036)	0.20678783404510548


In [117]:
vectorizer.get_feature_names_out()

array(['aby', 'acroparki', 'adrenalina', ..., 'żywo', 'żywy', 'żyć'],
      dtype=object)

In [116]:
print(vectors[0].toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [88]:
! pip install joblib

In [90]:
import joblib

joblib.dump(vectorizer, 'tfidf_vectorizer_wibit.joblib')

loaded_vectorizer = joblib.load('tfidf_vectorizer_wibit.joblib')

In [93]:
new_sample_text = "Jestem turystą, który uwielbia odwiedzać zamki i różnego rodzaju lochy, w szczególności sale tortur. Najchętniej jadam steki w renomowanych restauracjach"
prep_new_sample =  preprocess_text(new_sample_text)
print(prep_new_sample)

new_vector = loaded_vectorizer.transform([prep_new_sample])
print(new_vector)
print('')
new_vector = vectorizer.transform([prep_new_sample])
print(new_vector)

turysta który uwielbiać odwiedzać zamek i różny rodzaj loch w szczególność sala tortura chętnie jadać stek w renomowany restauracja
  (0, 2495)	0.24605789962083144
  (0, 2223)	0.17405551383888757
  (0, 2124)	0.19075797080157383
  (0, 2016)	0.4015333438899781
  (0, 1968)	0.4393755049638025
  (0, 1825)	0.38935088906906107
  (0, 1814)	0.1631576994206401
  (0, 1731)	0.27582440584758766
  (0, 1722)	0.2386689795910354
  (0, 1195)	0.17218351222358505
  (0, 789)	0.10676055700703477
  (0, 632)	0.37098128888572923
  (0, 191)	0.1690061969211308

  (0, 2495)	0.24605789962083144
  (0, 2223)	0.17405551383888757
  (0, 2124)	0.19075797080157383
  (0, 2016)	0.4015333438899781
  (0, 1968)	0.4393755049638025
  (0, 1825)	0.38935088906906107
  (0, 1814)	0.1631576994206401
  (0, 1731)	0.27582440584758766
  (0, 1722)	0.2386689795910354
  (0, 1195)	0.17218351222358505
  (0, 789)	0.10676055700703477
  (0, 632)	0.37098128888572923
  (0, 191)	0.1690061969211308


In [111]:
df.to_csv('oversample_stemmed_train_df.csv', index=False)

In [ ]:
# https://radimrehurek.com/gensim/models/doc2vec.html

In [113]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

VECTOR_SIZE = 100
MIN_COUNTS = 3
WINDOW = 5
EPOCHS = 100

tokenized_docs = [df['prep_text'][i].split(' ') for i in range(len(df))]
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(tokenized_docs)]

doc2vec_model = Doc2Vec(vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNTS, epochs=EPOCHS, seed=2023)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [114]:
doc2vec_model.save('doc2vec_model_wibit')
loaded_doc2vec_model = Doc2Vec.load('doc2vec_model_wibit')

In [115]:
new_sample_text = "Jestem turystą, który uwielbia odwiedzać zamki i różnego rodzaju lochy, w szczególności sale tortur. Najchętniej jadam steki w renomowanych restauracjach"
prep_new_sample =  preprocess_text(new_sample_text)
tokenized_new_sample = prep_new_sample.split(' ')

new_vector = doc2vec_model.infer_vector(tokenized_new_sample)
print(new_vector)

new_vector = loaded_doc2vec_model.infer_vector(tokenized_new_sample)
print(new_vector)

[ 0.19451219  1.1370583   0.42723057 -0.09534287 -0.54872936 -0.87539285
  0.31144127 -0.18335643  0.48054835 -0.55798554  0.31550604 -1.0254372
  0.17950983 -1.0452403   0.8924074  -0.5090677   0.9422483   0.4185485
  1.1656141  -0.6739846  -1.4971863  -1.3412588   0.7919278   0.6639775
 -0.2847138   1.0136237   1.201247    0.40899724 -0.1864483  -0.23833749
 -0.147709    0.04877814 -0.43337756  0.16722123 -0.4706023  -0.31890333
 -0.9621218   0.99878997 -0.05831239  1.5168557  -0.17033507  0.30291533
  1.1701149   1.0744908   0.49813068  0.00851204 -0.3712745  -0.16929272
  1.1261634  -0.12025094 -0.11904076 -0.31326774 -0.7928208  -1.0652969
  0.41152495  0.28458846 -0.54648983 -0.63744295  1.7932006   0.22744823
  0.9054576  -0.38440332 -0.0430477   0.6564841  -1.0138121  -0.3314889
  1.1564175  -0.03219074 -1.6682146  -0.39101917 -0.49299178  1.1708014
  1.2545644   0.78201246 -0.87722534 -0.03632323  0.35592118  1.002836
  0.20874898  0.30575967 -0.3348577   0.11902367 -0.2753809